Here is a brief outline of this chapter:
1. Common dictionary methods
2. Special handling for missing keys
3. Variations of dict in the standard library
4. The set and frozenset types
5. How hash tables work
6. Implications of hash tables (key type limitations, unpredictable ordering, etc.)

### Generic Mapping Types

![title](assets/chp3-umldiagram.png)

In [7]:
import collections.abc as abc
my_dict = {}
isinstance(my_dict, abc.Mapping)

True

Using isinstance is better than checking whether a function argument is of dict type, because then alternative mapping types can be used.

All mapping types in the standard library use the basic dict in their implementation, so they share the limitation that the keys must be hashable (the values need not be hashable, only the keys).

#### What Is Hashable?
An object is hashable if it contains a hash value which remains unchanged during its lifetime(it needs a `__hash()__` method) and can be compared with other methods(it needs an `__eq()__` method). Equal hashable objects ust have the same hash value.

In [8]:
tt = (1, 2, (30, 40))
hash(tt)

1350807749

In [9]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [10]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

1662783578

Note: 
`Python glossary states all immutable objects are hashable but that is inaccurate because tuples are immutable, yet it may contain references to unhashable objects`

Ways to build a dictionary:

In [14]:
a = dict(one=1, two=2, three=3)
b = {'one':1, 'two':2, 'three':3}
c = dict(zip(['one', 'two', 'three'],[1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

### Dict Comprehensions
The syntax of listcomps and genexps was applied to dictcomps and setcomps as well. Dictcomp builds a dict instance by producing key:value pair from any iterable.

In [15]:
DIAL_CODES = [
    (86, 'China'), 
    (91, 'India'), 
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]

country_code = {country:code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [18]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

The methods implemented by dict and two of its most useful variations: defaultdict and OrderedDict, both defined in the Collections module.
![title](assets/chp3-dict-default-ordered.png)

### Handling Missing Keys with setdefault
In line with the fail-fast philosophy, dict access with `d[k]` raises an error when k is not an existing key. Every Pythonista knows that `d.get(k, default)` is an alternative to `d[k]` whenever a default value is more convenient than handling KeyError. However, when updating the value found (if it is mutable), using either `__getitem__` or get is awkward and inefficient.
The example below shows that `dict.get`  is not the best way to handle missing keys.

In [24]:
import re

WORD_RE = re.compile('\w+')

index = {}
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            occurrences = index.get(word, [])
            occurrences.append(location)
            index[word] = occurrences

for word in sorted(index, key=str.upper):
    print(word, index[word])

Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9)]
by [(1, 20)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
Explicit [(4, 1)]
Flat [(7, 1)]
implicit [(4, 25)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6)]
nested [(7, 21)]
of [(1, 9)]
Peters [(1, 27)]
Python [(1, 12)]
Simple [(5, 1)]
than [(3, 21), (4, 20), (5, 18), (6, 19), (7, 16)]
The [(1, 1)]
Tim [(1, 23)]
ugly [(3, 26)]
Zen [(1, 5)]


The above does the folloing:
1. Takes text file as input.
2. Matches every word line by line. 
3. stores its location in a list. 
4. The location is a tuple formed by the line no and the column no. 
5. The index is a dict instance which stores the  words and the lists of location in the form of key-value pairs.

Here when storing the location inside the value of a key. It gets the key value using `index.get(word, [])`. 
The second argument is what it returns as the value of the the key when the key is not present in the dictionary.
We first store the value returned in a temporary variable, then append the location and finally store it in the dictionary.

The above process is too unoptimal.
The three lines of code can be replaced with a single line. This is where `setdefault` comes into picture. 

In [25]:
import re

WORD_RE = re.compile('\w+')

index = {}
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            index.setdefault(word, []).append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9)]
by [(1, 20)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
Explicit [(4, 1)]
Flat [(7, 1)]
implicit [(4, 25)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6)]
nested [(7, 21)]
of [(1, 9)]
Peters [(1, 27)]
Python [(1, 12)]
Simple [(5, 1)]
than [(3, 21), (4, 20), (5, 18), (6, 19), (7, 16)]
The [(1, 1)]
Tim [(1, 23)]
ugly [(3, 26)]
Zen [(1, 5)]


Using `setdefault` does the following:
1. Get the list of occurrences for word.
2. Set it to `[]` if not found. 
3. `setdefault` returns the value, so it can be updated without requiring a second search.

In [ ]:
my_dict.setdefault(key, []).append(new_value)

This is the same as running 

In [ ]:
if key not in my_dict:
 my_dict[key] = []
my_dict[key].append(new_value)

But the latter code performs at least two searches for key. Three if its not found. While setdefault does all this with only a single lookup.

### Mappings with Flexible Key Lookup

Sometimes it is convenient to have mappings that return some made-up value when missing key is searched. There are two main approaches:
1. Using a `defaultdict`.
2. Subclass `dict` or any other mapping type and add a `__missing__` method.

### `defaultdict`: Another Take on Missing Keys
Using collections.defaultdict provides a elegant solution for this problem. A defaultdict is configured to create items on demand whenever a missing key is searched. 

On instantiating a `defaultdict`, you  provide a callable that is
used to produce a default value whenever `__getitem__` is passed a nonexistent key argument.

On creating a default dict as `dd = defaultdict(list)`, when we call `dd['newkey']` and new key is non existent. Following steps are carried out:
1. Calls `list()` to create a list.
2. Inserts the list in the value of key : 'newkey'.
3. Returns a reference to the list.

The callable producing default values is held in an instance attribute the default_factory.

In [1]:
import re
import collections

WORD_RE = re.compile('\w+')

index = collections.defaultdict(list) #1
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            index[word].append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9)]
by [(1, 20)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
Explicit [(4, 1)]
Flat [(7, 1)]
implicit [(4, 25)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6)]
nested [(7, 21)]
of [(1, 9)]
Peters [(1, 27)]
Python [(1, 12)]
Simple [(5, 1)]
than [(3, 21), (4, 20), (5, 18), (6, 19), (7, 16)]
The [(1, 1)]
Tim [(1, 23)]
ugly [(3, 26)]
Zen [(1, 5)]


Here we initiate a defaultdict which creates an empty list when provided a non existent key.

So on calling `index[word].append(location)` it will always return reference to a list and append the location tuple in the list.

When no default_factory is provided the usual KeyError is raised.
The dedault_factory is only invoked to provide default values for `__getitem__` calls. So if k is missing `dd.get(k)` (dd is a default dict) still returns `none` but a call to default factory is ade for creating default value.

### The `__missing__` Method

This method is the way mappings deal with missing keys. It is not defined in base dict class but dict ias aware of it. If we subclass dict and provide a `__missing__` method. The `dict.__getitem__` will call it on the missing key instead of raising KeyError.

This method is only used by the `__getitem__` call and hence `default_factory` of default dict only works with it.

In [32]:
class StrKeyDict0(dict):  # <1>

    def __missing__(self, key):
        if isinstance(key, str):  # <2>
            raise KeyError(key)
        return self[str(key)]  # <3>

    def get(self, key, default=None):
        try:
            return self[key]  # <4>
        except KeyError:
            return default  # <5>

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() # <6>
    
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

1. `StrKeyDict0` inherits from `dict`.
2. Check whether key is already a `str`. If it is, and it’s missing, raise `KeyError`.

If the `isinstance` was not used, it would result in an infinite recursion if the `str(key)` would be non existent.

3. Build str from key and look it up.
4. The get method delegates to `__getitem__` by using the `self[key]` notation; that gives the opportunity for our `__missing__` to act.
5. If a `KeyError` was raised, `__missing__` already failed, so we return the default.
6. Search for unmodified key (the instance may contain non-str keys), then for a str built from the key.

In [33]:
d[4]

'four'

In [6]:
d['5']

KeyError: '5'

In [7]:
d.get(2)

'two'

In [8]:
d.get('4')

'four'

In [9]:
d.get('5','N/A')

'N/A'

In [10]:
2 in d

True

In [11]:
1 in d

False

Since the operation k in d calls the `__contains__` method we have to change the implementation as the dict class does not fall back to invoking `__missing__`

There is a subtle detail in our implementation of `__contains__`: we do not check for the key in the usual Pythonic way —> k in my_dict because `str(key) in self` would recursively call `__contains__`. We avoid this by explicitly looking up the key in `self.keys()`.

A search like k in `my_dict.keys()` is efficient in Python 3 even
for very large mappings because `dict.keys()` returns a view,
which is similar to a set, and containment checks in sets are as
fast as in dictionaries.

#### Variations of dict

Let's summarize the different mapping types in the collections module besides default dict:

**`collections.OrderedDict`**: Maintains key in insertion order. The **`popitem`** method pops the first item by default, and last if `ord_dict.popitem(last=True)`.

**`collections.ChainMap`**: Chainmap holds a list of mappings which can be searched as one. The lookup is performed in order and is success if key is found in any one of them.

**`collections.Counter`**: A mapping which holds an integer count for each key. Updating an existing key adds to its count. Counter implements the `+` and `-` operators to combine tallies, and other useful methods such as `most_common([n])`, which returns an ordered list of tuples with the n most common items and their counts.

For eg,

In [15]:
import collections
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [16]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 3})

In [17]:
ct.most_common(2)

[('a', 10), ('z', 3)]

**`collections.UserDict`**: A pure python implementation of a mapping that works like a standard dict.

While OrderedDict, ChainMap, and Counter come ready to use, UserDict is designed to be subclassed.

### Subclassing UserDict
It’s almost always easier to create a new mapping type by extending UserDict rather than dict. Its value can be appreciated as we extend our StrKeyDict0 from to make sure that any keys added to the mapping are stored as str.

Note that UserDict does not inherit from dict, but has an internal dict instance, called `data`, which holds the actual items. This avoids undesired recursion when coding special methods like `__setitem__`, and simplifies the coding of `__contains__`.

Due to UserDict, the implementation StrKeyDict is shorter than the original StfKeyDict0 which inherited dict but it does more: it stores all keys as str, avoiding unpleasant surprises if he instance is built or updated with data containing nonstring keys. 

In [20]:
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

`__contains__` is simpler: we can assume allstored keys are str and we can check on `self.data` instead of invoking `self.keys()` as we did in `StrKeyDict0`.

`__setitem__` converts any key to str before writing to `self.data` attribute.

Because UserDict subclasses MutableMapping, the remaining methods that make
StrKeyDict a full-fledged mapping are inherited from UserDict, MutableMapping, or Mapping. The latter have several useful concrete methods, in spite of being abstract base classes (ABCs). The following methods are worth noting:

**`MutableMapping.update`**
This is a powerful method also used by `__init__` to load instance from other mappings, from iterables of (key, value) pairs andkeyword arguments. Since, it uses `self[key] = value` to add items, it ends up calling `__setitem__`.

**`Mapping.get`**

### Immutable Mappings
The mapping types provided by the standard library are all mutable, but you may need to guarantee that a user cannot change a mapping by mistake.

Since Python 3.3, the types module provides a wrapper class called `MappingProxyType`, which, given a mapping, returns a mappingproxy instance that is a read-only but dynamic view of the original mapping. This means that updates to the original mapping can be seen in the mappingproxy, but changes cannot be made through it.

In [21]:
from types import MappingProxyType

d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [22]:
d_proxy[1]

'A'

In [23]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [25]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [26]:
d_proxy[2]

'B'

1. Items in `d` can be seen through `d_proxy`.
2. Changes cannot be made through `d_proxy`.
3. `d_proxy` is dynamic, any change in `d` is reflected.

### Set Theory
Sets are relatively new in python. A set is a collection of unique objects.A basic use case is removing duplication.


In [28]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)

{'eggs', 'spam'}

In [29]:
list(set(l))

['spam', 'eggs']

 Set elements must be hashable. The set type is not hashable, but frozenset is, so you can have frozenset elements inside set.
 Set also implements essential set operations as infix operators.
 1. `A | B` - Union
 2. `A & B` - Intersection
 3. `A - B` - Difference

Sets can reduce both the code lines and the program runtime by removubg loops and conditional logic.
 
For eg, you need to count how many email addreses occur froma small set of email addresses(needles) in a large set of email addresses(haystack).
 
Using set:

In [ ]:
found = len(needles & haystack)

Using loops:

In [ ]:
found = 0
for n in needles:
    if n in haystack:
        found += 1


The set example is faster than the loop example but it requires that both needles and haysack be sets. Whereas, loops can be used over any iterable.

But remember, u can build sets on the fly.

In [ ]:
found = len(set(needles) & set(haystack))
found = len(set(needles).intersection(haystack))

Any one of the preceding examples are capable of searching 1,000 values in a haystack of 10,000,000 items in a little over 3 milliseconds—that’s about 3 microseconds per needle.

Besides the extremely fast membership test (thanks to the underlying hash table), the `set` and `frozenset` built-in types provide a rich selection of operations to create new sets or, in the case of set, to change existing ones. 

### `set` Literals

The syntax of `set` literals—{1}, {1, 2}, etc.—looks exactly like the math notation, with one important exception: there’s no literal notation for the empty set, so we must remember to write `set()`.


In [34]:
s = {1}
type(s)

set

In [35]:
s

{1}

In [36]:
s.pop()

1

In [37]:
s

set()

Literal set syntax like {1, 2, 3} is both faster and more readable than calling the constructor (e.g., `set([1, 2, 3]))`. The latter form is slower because, to evaluate it, Python has to look up the set name to fetch the constructor, then build a list, and finally pass it to the constructor. In contrast, to process a literal like {1, 2, 3}, Python runs a specialized `BUILD_SET` bytecode.


In [38]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [39]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


Three operations instead of `BUILD_SET`: `LOAD_NAME`, `BUILD_LIST` and
`CALL_FUNCTION`.

There is no special syntax to represent `frozenset` literals—they must be created by calling the constructor. The standard string representation in Python 3 looks like a `frozenset` constructor call.

In [40]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

### Set Comprehensions

In [42]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

Build set of characters with codes from 32 to 255 that have the word `'SIGN'` in their names.

### Set Operations
Many of the set operations are special methods for operator overloading.
Some operators and methods perform in-place changes on the target set (e.g., `&=`, `difference_update`, etc.). Such operations make no sense in the ideal world of mathematical sets, and are not implemented in `frozenset`.

![title](assets/chp3-umlset.png)

Math operations for set:
![title](assets/chp3-setmath.png)



### `dict` and `set` Under the Hood
Understanding how Python dictionaries and sets are implemented using hash tables is helpful to make sense of their strengths and limitations.

Here are some questions this section will answer:
1. How efficient are Python dict and set?
2. Why are they unordered?
3. Why can’t we use any Python object as a dict key or set element?
4. Why does the order of the dict keys or set elements depend on insertion order, and may change during the lifetime of the structure?
5. Why is it bad to add items to a dict or set while iterating through it?

#### A Performance Experiment

To see how the size of a `dict`, `set`, or `list` affects the performance of search using the in operator, I generated an array of 10 million distinct double-precision floats, the “haystack.” I then generated an array of needles: 1,000 floats, with 500 picked from the haystack and 500 verified not to be in it.

![title](assets/chp3-benchmark.png)

For the dict benchmark, `dict.fromkeys()` is used to create a dict named haystack with 1,000 floats. 
All the benchmarks use the `in` operator except the `set&` which uses the
`'&'` intersection method from set.
The `set&` benchmark is the best among all the others.
Whereas, in list you can see the time rising linearly as the haystack size increases as it has to do a full list scan every time.


### Hash Tables in Dictionaries
This is a high-level view of how Python uses a hash table to implement a dict.
A hash table is a sparse array (i.e., an array that always has empty cells). In standard data structure texts, the cells in a hash table are often called “buckets.” In a dict hash table, there is a bucket for each item, and it contains two fields: a reference to the key and a reference to the value of the item. Because all buckets have the same size, access to an individual bucket is done by offset.

Python tries to keep at least 1/3 of the buckets empty; if the hash table becomes too crowded, it is copied to a new location with room for more buckets.
To put an item in a hash table, the first step is to calculate the hash value of the item key, which is done with the `hash()` built-in function

The `hash()` built-in function works directly with built-in types and falls back to calling `__hash__` for user-defined types. If two objects compare equal, their hash values must also be equal, otherwise the hash table algorithm does not work. 

Starting with Python 3.3, a random salt value is added to the
hashes of `str`, `bytes`, and `datetime` objects. The salt value is
constant within a Python process but varies between interpreter
runs. The random salt is a security measure to prevent a DOS
attack

#### The hash table algorithm

To fetch value at `my_dict[search_key]`, python calls hash(search_key) to obtain hash value of search_key and uses the lsb's as offset to lookup a bucket in the hash table. If the found bucket is empty, `KeyError` is raised. Otherwise, the found bucket has an item—a `found_key:found_value` pair—and then Python checks whether `search_key == found_key`. If they match, that was the item sought: `found_value` is returned.

However, if `search_key` and `found_key` do not match, this is a hash collision. This happens because a hash function maps arbitrary objects to a small number of bits, and—in addition—the hash table is indexed with a subset of those bits. In order to resolve the collision, the algorithm then takes different bits in the hash, massages them in a particular way, and uses the result as an offset to look up a different bucket. If that is empty, `KeyError` is raised; if not, either the keys match and the item value is returned, or the
collision resolution process is repeated.

![title](assets/chp3-hashalgorithm.png)

The process to insert or update an item is the same, except that when an empty bucket is located, the new item is put there, and when a bucket with a matching key is found, the value in that bucket is overwritten with the new value.

Additionally, when inserting items, Python may determine that the hash table is too crowded and rebuild it to a new location with more room. As the hash table grows, so does the number of hash bits used as bucket offsets, and this keeps the rate of collisions low.

This implementation may seem like a lot of work, but even with millions of items in a dict, many searches happen with no collisions, and the average number of collisions per search is between one and two. Under normal usage, even the unluckiest keys can be found after a handful of collisions are resolved. 

Knowing the internals of the dict implementation we can explain the strengths and limitations of this data structure and all the others derived from it in Python.

### Practical Consequences of how `dict` works

#### Keys must be hashable objects
An object is hashable only if:
1. It supports `hash()` function via the `hash()` method that returns same value over the lifetime of the object.
2. Must support equality via `eq()` method.
3. If `a == b` is `True` then `hash(a) == hash(b)` must also be `True`.

User-defined types are hashable by default because their hash value is their `id()` and they all compare not equal.

If you implement a class with a custom `__eq__` method, you must
also implement a suitable `__hash__`, because you must always make
sure that if `a == b` is True then `hash(a) == hash(b)` is also True.
Otherwise you are breaking an invariant of the hash table algo‐
rithm, with the grave consequence that dicts and sets will not deal
reliably with your objects. If a custom `__eq__` depends on muta‐
ble state, then `__hash__` must raise `TypeError` with a message like
unhashable type: `'MyClass'`.

#### dicts have significant memory overhead
Because a `dict` uses a hash table internally, and hash tables must be sparse to work, they are not space efficient. 
Replacing dicts with tuples reduces the memory usage in two ways: by removing the overhead of one hash table per record and by not storing the field names again with each record.

#### Key search is very fast
The `dict` implementation is an example of trading space for time: dictionaries have significant memory overhead, but they provide fast access regardless of the size of the dictionary—as long as it fits in memory.

#### Key ordering depends on insertion order
When a hash collision happens, the second key ends up in a position that it would not normally occupy if it had been inserted first.

In [5]:
DIAL_CODES = [
     (86, 'China'),
     (91, 'India'),
     (1, 'United States'),
     (62, 'Indonesia'),
     (55, 'Brazil'),
     (92, 'Pakistan'),
     (880, 'Bangladesh'),
     (234, 'Nigeria'),
     (7, 'Russia'),
     (81, 'Japan'),
 ]
d1 = dict(DIAL_CODES)
print('d1:', d1.keys())


d1: dict_keys([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])


In [6]:
d2 = dict(sorted(DIAL_CODES))
print('d2', d2.keys())

d2 dict_keys([1, 7, 55, 62, 81, 86, 91, 92, 234, 880])


In [7]:
d3 = dict(sorted(DIAL_CODES, key = lambda x:x[1]))
print('d3:', d3.keys())

d3: dict_keys([880, 55, 86, 91, 62, 81, 234, 92, 7, 1])


In [9]:
assert d1 == d2 and d2 == d3

1. **d1**: built from the tuples in descending order of country population.
2. **d2**: built from the tuples sorted by dial code.
3. **d3**: loaded with tuples sorted by country name.
4. The dictionaries compares equal, because they hold the same key:value pairs. 

#### Adding items may change the order of items.

When adding a new item to a dict, the interpreter may decide that the hash table needs to grow. During this process, new hash collisions may happen, so the result is likely that the items are ordered differently in the new hash table. This is implementation dependent and cannot be predicted reliably. That is why modifying contents of a dict while iterating is a bad idea, because the dictionary may not scan all the items if in case python decides to reorder the hash table.

Thus, the correct way is to scan the dict from start to finish and collect the needed additions in second dict and update it with the first one when finished.

### How Sets Work

The `set` and `frozenset` types are also implemented using a hash table except that each bucket holds only a reference to the element.

1. Set elements must be hashable objects.
2. Sets have significant memory overhead.
3. Mmebership testing is very efficient.
4. Element ordering depends on insertion order.
5. Adding elements to a set may change the order of the other elements.

